# ML Pipeline

In [1]:
# imports
import pandas as pd
import numpy as np
import random

from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [40]:
# read in df
name = '2024.04.24_Mastertabelle_ML.xlsx'
path_win = 'G://My Drive//Forschung//Mitarbeiter//Allgaier//23-12-06_Immun-ML//04_Data//03_ML//'
path_mac = '/Users/johannesallgaier/My Drive/Forschung/Mitarbeiter/Allgaier/23-12-06_Immun-ML/04_Data/03_ML'

df = pd.read_excel(path_win + name, index_col='Unnamed: 0')

## Prepare

In [41]:
# drop rows where the last event did not happen today
df = df[df['days_since_last_event']>0]

In [42]:
# create new feature
df['n_events_so_far'] = df['n_vaccinations_so_far'] + df['n_infections_so_far']

In [43]:
def add_days_diff(df):
    """
    Adds a 'days_diff' column to the DataFrame which calculates the number of days
    between subsequent dates for each patient ID.

    Parameters:
    df (pd.DataFrame): A pandas DataFrame with columns 'ID' and 'date'.

    Returns:
    pd.DataFrame: A new DataFrame with the 'days_diff' column added.
    """
    # Ensure 'date' column is of datetime type
    df['date'] = pd.to_datetime(df['date'])

    # Sort the dataframe by 'ID' and 'date'
    df = df.sort_values(by=['ID', 'date'])

    # Calculate the future 'SARS-IgG' value and assign to 'SARS-IgG_future_value' column
    df['SARS-IgG_future_value'] = df.groupby('ID')['SARS-IgG'].shift(-1)

    # Calculate the difference in days to the next date and assign to 'days_diff' column
    df['days_until_next_measurement'] = df.groupby('ID')['date'].shift(-1) - df['date']
    df['days_until_next_measurement'] = df['days_until_next_measurement'].dt.days

    return df

df = add_days_diff(df)
# check function
df[['ID','date', 'SARS-IgG', 'days_until_next_measurement', 'SARS-IgG_future_value']].head(15)

,ID,date,SARS-IgG,days_until_next_measurement,SARS-IgG_future_value
2,C1,2021-03-12,18.27,30.0,247.59
3,C1,2021-04-11,247.59,60.0,91.35
4,C1,2021-06-10,91.35,90.0,52.08
5,C1,2021-09-08,52.08,65.0,1484.70
7,C1,2021-11-12,1484.70,60.0,711.90
8,C1,2022-01-11,711.90,90.0,360.99
9,C1,2022-04-11,360.99,90.0,321.22
10,C1,2022-07-10,321.22,90.0,1311.04
11,C1,2022-10-08,1311.04,90.0,1905.27
12,C1,2023-01-06,1905.27,90.0,1245.23


In [44]:
# define feature list
features = ['SARS-IgG', 'Alter', 'Geschlecht', 'Dialyse_x',
            'n_events_so_far', 'vaccination', 'days_until_next_measurement']

In [45]:
# define new target
target = ['SARS-IgG_future_value']

## Define train and test users

In [46]:
# define train and test users
random.seed(1994)

all_users = df['ID'].unique().tolist()
train_users = random.sample(all_users, int(len(all_users)*0.8))
test_users = [user for user in all_users if user not in train_users]

assert set(train_users + test_users) == set(all_users)

In [47]:
# option 1 - train and test set (nested cv)
# define train and test dataframes
# df_train = df[df['ID'].isin(train_users)]
# df_test = df[df['ID'].isin(test_users)]
# assert df_train.shape[0] + df_test.shape[0] == df.shape[0]

# option 2 - standard cv
df_train = df

## Transform

In [10]:
# Alter
"""
bins = np.arange(0, 120, 10)
df_train.loc[:,'Alter_bins'] = pd.cut(df_train.Alter, bins)

le_age = LabelEncoder()
df_train.loc[:,'Alter_bins'] = le_age.fit_transform(df_train['Alter_bins'].values)
"""

In [11]:
#SARS-IgG
"""
rs = RobustScaler()
rs.fit(df_train['SARS-IgG'].values.reshape(-1, 1))
df_train.loc[:, 'SARS-IgG'] = rs.transform(df_train.loc[:,'SARS-IgG'].values.reshape(-1, 1))
"""

## Group Cross Validate

In [12]:
# define features and target
X = df_train[features].dropna()
y = df_train.loc[X.index, target].values.ravel()

total_rmse_scores=list()

# Retrieve an identifier column 'ID' from the dataframe
groups = df_train.loc[X.index, 'ID']

for rs in range(100):

    # Initialize a Lasso regression model.
    model = linear_model.LinearRegression()
    # Perform 5-fold group cross-validation to evaluate model using negative MSE.
    total_rmse_scores.extend(list(cross_val_score(model, X, y, groups=groups,cv=5, scoring='neg_mean_squared_error')))

# Convert MSE scores to RMSE scores.
rmse_scores = [(-1*score)**0.5 for score in total_rmse_scores]
print('mean RMSE:\t', int(np.array(rmse_scores).mean()))
print('std RMSE:\t', int(np.array(rmse_scores).std()))

#TODO: go to hold-out test set


mean RMSE:	 2658
std RMSE:	 892


### Performance Log of Mean RMSE (Std):
```
features = ['Alter', 'Geschlecht', 'Dialyse_x', 'n_events_so_far','SARS-IgG', 'vaccination']
Mean    2329
Lasso   2525
RF      2220 (495)
RF      2599 (340)
SVR     2446 (500)
RF (default params)     2545 (339)
Adaboost    2642 (327)

features = ['Alter', 'Geschlecht', 'Dialyse_x', 'n_vaccinations_so_far', 'n_infections_so_far','SARS-IgG', 'vaccination']
RF  2508 (227)

--- with new days_diff approach ---
nested approach:
RF  2800 (300)

no nested approach (standard cv)
RF  2846 (702)
RF with n_trees = n_features 3058 (859)

```


## How good is a simple heuristic? (Grouped by days diff?)

In [48]:
bins = [-1, 15, 30, 90, 180, 365, 365*2, 365*3]
df['days_until_next_measurement_bins'] = pd.cut(df.days_until_next_measurement, bins=bins)
res = df.groupby(['days_until_next_measurement_bins', 'Dialyse_x'], observed=False)['SARS-IgG_future_value'].describe()
res.to_excel('../../results/heuristic_by_days_and_dialyse.xlsx')
res = res.reset_index()
res


,days_until_next_measurement_bins,Dialyse_x,count,mean,std,min,25%,50%,75%,max
0,"(-1, 15]",1,7.0,1592.665714,3028.391130,20.79,122.190,382.620,1064.7500,8371.37
1,"(15, 30]",0,54.0,981.306667,932.940342,57.12,373.275,757.630,1303.7200,4565.40
2,"(15, 30]",1,295.0,3439.456847,6269.564122,6.30,308.910,1269.850,4074.7700,55047.60
3,"(30, 90]",0,427.0,884.808665,1180.407294,6.30,171.150,405.300,1130.4800,7835.74
4,"(30, 90]",1,875.0,1251.347669,2391.202782,6.30,100.365,423.780,1429.3850,36876.43
5,"(90, 180]",0,91.0,1482.373077,1311.606407,12.39,634.115,1245.300,1838.5500,8947.68
6,"(90, 180]",1,38.0,2889.695526,3954.233962,6.30,584.850,1577.065,2917.3975,18207.44
7,"(180, 365]",0,46.0,1805.760000,2310.622144,118.93,537.985,863.740,2153.6925,10160.64
8,"(180, 365]",1,1.0,2438.850000,NaN,2438.85,2438.850,2438.850,2438.8500,2438.85
9,"(365, 730]",0,3.0,5072.016667,8038.264989,368.21,431.265,494.320,7423.9200,14353.52


In [49]:
def find_mean_value(df, number, binary_value, interval_column, binary_column, mean_column):
    """
    Finds the mean value from the DataFrame where the given number falls within the interval
    and the binary column matches the binary value.

    Parameters:
    df (pd.DataFrame): A pandas DataFrame with interval, binary, and mean columns.
    number (float): The number to check which interval it falls into.
    binary_value (int): The binary value to match in the binary column.
    interval_column (str): The name of the interval column.
    binary_column (str): The name of the binary column.
    mean_column (str): The name of the mean value column.

    Returns:
    float: The mean value where the number falls within the interval and the binary value matches.
    """
    # Convert the interval column to an IntervalIndex
    intervals = pd.IntervalIndex(df[interval_column])

    # Find the index where the number falls within the interval
    index = intervals.contains(number)

    # Filter the DataFrame by the interval index and binary value
    filtered_df = df[index & (df[binary_column] == binary_value)]

    # Return the corresponding mean value
    if not filtered_df.empty:
        return filtered_df[mean_column].values[0]
    else:
        return None


In [50]:
def apply_heuristic_mean_prediction(main_df, interval_df, interval_column, binary_column, mean_column):
    """
    Applies the heuristic mean prediction to the main DataFrame.

    Parameters:
    main_df (pd.DataFrame): The main DataFrame with 'days_until_next_measurement' and 'Dialyse_x' columns.
    interval_df (pd.DataFrame): The interval DataFrame with interval, binary, and mean columns.
    interval_column (str): The name of the interval column in the interval DataFrame.
    binary_column (str): The name of the binary column in both DataFrames.
    mean_column (str): The name of the mean value column in the interval DataFrame.

    Returns:
    pd.DataFrame: The main DataFrame with an added 'heuristic_mean_prediction' column.
    """
    predictions = main_df.apply(
        lambda row: find_mean_value(interval_df, row['days_until_next_measurement'], row[binary_column], interval_column, binary_column, mean_column),
        axis=1
    )
    main_df['heuristic_mean_prediction'] = predictions
    return main_df

In [55]:
df

,date,ID,vaccination,timepoint,SARS-IgG,infection,Dialyse_x,n_vaccinations,n_infections,group_id,...,future_measurement_date,future_measurement_val,Alter,Geschlecht,Dialyse_y,n_events_so_far,SARS-IgG_future_value,days_until_next_measurement,days_until_next_measurement_bins,heuristic_mean_prediction
2,2021-03-12,C1,0.0,T1,18.27,0,0,2,0,2,...,NaN,NaN,61,1,0,1.0,247.59,30.0,"(15.0, 30.0]",981.306667
3,2021-04-11,C1,0.0,T2,247.59,0,0,2,0,2,...,2022-04-11,360.99,61,1,0,1.0,91.35,60.0,"(30.0, 90.0]",884.808665
4,2021-06-10,C1,0.0,T3,91.35,0,0,2,0,2,...,NaN,NaN,61,1,0,1.0,52.08,90.0,"(30.0, 90.0]",884.808665
5,2021-09-08,C1,0.0,T4,52.08,0,0,2,0,2,...,NaN,NaN,61,1,0,1.0,1484.70,65.0,"(30.0, 90.0]",884.808665
7,2021-11-12,C1,0.0,T12,1484.70,0,0,2,0,2,...,NaN,NaN,61,1,0,2.0,711.90,60.0,"(30.0, 90.0]",884.808665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3364,2023-04-04,HD98,0.0,T34,8697.80,0,1,5,1,9,...,NaN,NaN,76,0,1,6.0,NaN,NaN,NaN,NaN
3367,2021-02-28,HD99,0.0,T2,51.66,0,1,3,0,7,...,NaN,NaN,80,1,1,2.0,20.79,60.0,"(30.0, 90.0]",1251.347669
3368,2021-04-29,HD99,0.0,T3,20.79,0,1,3,0,7,...,NaN,NaN,80,1,1,2.0,6.30,90.0,"(30.0, 90.0]",1251.347669
3369,2021-07-28,HD99,0.0,T4,6.30,0,1,3,0,7,...,NaN,NaN,80,1,1,2.0,237.30,112.0,"(90.0, 180.0]",2889.695526


In [51]:
result_df = apply_heuristic_mean_prediction(df, res, 'days_until_next_measurement_bins', 'Dialyse_x', 'mean')

In [52]:
y_pred = result_df.dropna(subset='SARS-IgG_future_value')['heuristic_mean_prediction']
y_true = result_df.dropna(subset='heuristic_mean_prediction')['SARS-IgG_future_value']

from sklearn.metrics import mean_squared_error
print('RMSE\t', int(mean_squared_error(y_true, y_pred)**0.5))

RMSE	 3160


Unexpected result of 3160. I thought it would decrease if we differentiate by prediction horizon. It's not yet clear whether there is a bug in the code or in the data.

In [65]:
result_df[['ID','date', 'Dialyse_x', 'SARS-IgG','days_until_next_measurement', 'SARS-IgG_future_value', 'heuristic_mean_prediction',]]

,ID,date,Dialyse_x,SARS-IgG,days_until_next_measurement,SARS-IgG_future_value,heuristic_mean_prediction
2,C1,2021-03-12,0,18.27,30.0,247.59,981.306667
3,C1,2021-04-11,0,247.59,60.0,91.35,884.808665
4,C1,2021-06-10,0,91.35,90.0,52.08,884.808665
5,C1,2021-09-08,0,52.08,65.0,1484.70,884.808665
7,C1,2021-11-12,0,1484.70,60.0,711.90,884.808665
...,...,...,...,...,...,...,...
3364,HD98,2023-04-04,1,8697.80,NaN,NaN,NaN
3367,HD99,2021-02-28,1,51.66,60.0,20.79,1251.347669
3368,HD99,2021-04-29,1,20.79,90.0,6.30,1251.347669
3369,HD99,2021-07-28,1,6.30,112.0,237.30,2889.695526
